In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
import shap
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, auc, precision_score, recall_score, precision_recall_curve
np.random.seed(42)

In [2]:
codigos_ibge_estados = [
    12, 27, 16, 13, 29, 23, 53, 32, 52, 21, 51, 50,
    31, 15, 25, 41, 26, 22, 33, 24, 43, 11, 14, 42, 28, 35, 17
]
list_res = []

In [3]:
for sg_num in codigos_ibge_estados:
    print(sg_num)
    name_lab = 'data/lab_'+str(sg_num)+'.csv'
    name_epi = 'data/epi_'+str(sg_num)+'.csv'
    df_lab = pd.read_csv(name_lab)
    df_epi = pd.read_csv(name_epi)
    df_lab = df_lab.drop(columns = ['ID_MN_RESI'])
    df_epi = df_epi.drop(columns = ['ID_MN_RESI'])
    denv_lab = sum(df_lab['CHIK'] == 0) 
    chik_lab = sum(df_lab['CHIK'] == 1) 
    denv_epi = sum(df_epi['CHIK'] == 0) 
    chik_epi = sum(df_epi['CHIK'] == 1) 
    df_lab = pd.get_dummies(df_lab, columns = ['fx_etaria'])
    df_epi = pd.get_dummies(df_epi, columns = ['fx_etaria'])
    df_epi = df_epi.reindex(columns = df_lab.columns, fill_value=0)
    df_lab = df_lab.drop_duplicates()
    X_lab = df_lab.drop(columns = ['CHIK'])
    y_lab = df_lab['CHIK']
    X_epi = df_epi.drop(columns = ['CHIK'])
    y_epi = df_epi['CHIK']
    X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, stratify=y_lab, test_size = 0.3, random_state = 0)
    model = XGBClassifier(nthread = 8)
    model.fit(X_train, y_train)
    y_train_prob = model.predict_proba(X_train)[:,1]
    y_test_prob = model.predict_proba(X_test)[:,1]
    auc_train = roc_auc_score(y_train, y_train_prob)
    auc_test = roc_auc_score(y_test, y_test_prob)
    fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)
    distances = np.sqrt((1 - tpr)**2 + (fpr)**2)
    best_threshold = thresholds[np.argmin(distances)]
    y_epi_corr = model.predict_proba(X_epi)[:,1]
    denv_epi_corr = sum(y_epi_corr < best_threshold)
    chik_epi_corr = sum(y_epi_corr >= best_threshold)
    list_res.append([sg_num, denv_lab, chik_lab, denv_epi, chik_epi, auc_train, auc_test, denv_epi_corr, chik_epi_corr])

12
27
16
13
29
23
53
32
52
21
51
50
31
15
25
41
26
22
33
24
43
11
14
42
28
35
17


In [4]:
df_res = pd.DataFrame(list_res)

In [5]:
df_res.columns = ['SG_UF', 'Denv Lab', 'Chik Lab', 'Denv Epi', 'Chik Epi', 'AUC Treino', 'AUC Teste', 'Denv Epi Corr', 'Chik Epi Corr']

In [6]:
df_res = df_res.sort_values(by = 'SG_UF')

In [7]:
df_res = df_res.reset_index(drop = True)

In [8]:
df_res

,SG_UF,Denv Lab,Chik Lab,Denv Epi,Chik Epi,AUC Treino,AUC Teste,Denv Epi Corr,Chik Epi Corr
0,11,1948,143,1733,0,0.995383,0.825440,1518,215
1,12,1060,156,1535,3,0.981187,0.664793,982,556
2,13,2456,13,2825,2,0.997028,0.759732,954,1873
3,14,195,2,33,0,0.887500,0.509615,10,23
4,15,7031,202,5550,47,0.999419,0.900437,4152,1445
5,16,2606,175,2595,1,0.990816,0.830016,2150,446
6,17,1311,167,1914,101,0.999125,0.821754,1388,627
7,21,1723,316,4933,297,0.997421,0.826156,3174,2056
8,22,2670,164,6145,275,0.998678,0.897612,2629,3791
9,23,3678,478,4097,110,0.990774,0.773869,2889,1318
